# Environments

# Access AWS from local machine

First and foremost, add AWS access keys to env variables to local machine, ideally in `~/.zsh` (or whatver your $SHELL is).
```bash
export AWS_ACCESS_KEY_ID="..."
export AWS_SECRET_ACCESS_KEY="..."
```
these can be found in the AWS console under IAM -> Users -> Security credentials.

# Start/stop an instance

```bash
aws ec2 start-instances --instance-ids i-0a92dcb7d252071c5
aws ec2 stop-instances --instance-ids i-0a92dcb7d252071c5
```

or via IP address:
```bash
aws ec2 start-instances --instance-ids $(aws ec2 describe-instances --filters "Name=ip-address,Values=54.151.70.169" --query 'Reservations[*].Instances[*].InstanceId' --output text)
aws ec2 stop-instances --instance-ids $(aws ec2 describe-instances --filters "Name=ip-address,Values=54.151.70.169" --query 'Reservations[*].Instances[*].InstanceId' --output text)
```


## ssh and connect

Optionally, create and assign elastic IP to the instance (cost is $3.5 per month) for easier access.

If you have a pem key you can just log in with:
```bash
ssh -i my-aws-gpu-machine.pem ubuntu@54.151.70.169
```
or if you make a ~/.ssh/config file like this:
```bash
Host my-aws-gpu-instance 
  HostName 54.151.70.169
  User ubuntu
  IdentityFile ~/.ssh/my-aws-gpu-instance.pem   # if you lose this you can use the trick below
```
then just `ssh my-aws-gpu-instance`.

If you happen to lose pem file, just:
- create a key pair run this on LOCAL machine (skip this if already exists and just copy it: `cat ~/.ssh/id_rsa.pub`)
    ```bash
    ssh-keygen -t rsa -b 4096 -C "<pick a name of a machine you are running this command on>"
    ```
- log into remote machine using ec2 instance connect
- copy LOCAL public key to `authorized_keys`:
`echo public_key_string >> ~/.ssh/authorized_keys`
or use `nano` editor.
- change permissions on REMOTE machine:
    ```bash
    chmod 700 ~/.ssh
    chmod 600 ~/.ssh/authorized_keys
    ```

# Change shell prompt appearance

Open `~/.bashrc` and add the following lines:
```bash
# Function to get public IP address
get_public_ip() {
    curl -s ifconfig.me
}

# Set the PROMPT variable to include the public IP address
PROMPT='%n@$(get_public_ip) %~ %# '
```


# Access via VSCode

On the left just add remote instance, or Cmd+Shift+P and type `Remote-SSH: Connect to Host...` and type `ssh my-aws-gpu-instance`.


## Add github access

Take a public key from any machine and copy/paste into GitHub - Settings - SSH and GPG keys - New SSH key.



## Anything GPU


To see recommended drivers:
```bash
sudo apt install ubuntu-drivers-common
ubuntu-drivers devices
```
for T4 GPU, the recommended driver is 440. To install the latest driver, run the following commands:
```bash
sudo apt update
sudo apt-get upgrade   # conservative upgrade
sudo apt-get dist-upgrade   # aggressive upgrade, be careful
sudo apt install nvidia-driver-545
sudo reboot  # don't forget to reboot, might even need to purge old drivers via `sudo apt-get purge nvidia-driver-XXX`
```
You can now see stats:
```bash
lspci | grep -i nvidia
nvidia-smi
```
you should see something like this:
```
NVIDIA-SMI 545.29.06              Driver Version: 545.29.06    CUDA Version: 12.3    
```


# Install any Python on Linux using pyenv

You do this only once:
```bash
# Install pyenv prerequisites
sudo apt update
sudo apt install -y make build-essential libssl-dev zlib1g-dev libbz2-dev \
libreadline-dev libsqlite3-dev wget curl llvm libncurses5-dev libncursesw5-dev \
xz-utils tk-dev libffi-dev liblzma-dev python3-openssl git

# Install pyenv
curl https://pyenv.run | bash

# Add pyenv to the shell
echo 'export PATH="$HOME/.pyenv/bin:$PATH"' >> ~/.bashrc
echo 'eval "$(pyenv init --path)"' >> ~/.bashrc
echo 'eval "$(pyenv virtualenv-init -)"' >> ~/.bashrc
source ~/.bashrc

# Install Python 3.10.14
pyenv install 3.10.14
```

Set global Python version:
```bash
pyenv global 3.10.14
```
or local:
```bash
pyenv local 3.10.14
```


Also have option to add to the PYTHONPATH:
```bash
export PYTHONPATH=$PYTHONPATH:$PWD 
```
github username can also come handy:
```bash
export GITHUB_USERNAME="nesaboz"
```

## Install AWS CLI

```bash
sudo apt install unzip
curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
unzip awscliv2.zip
sudo ./aws/install
source ~/.bashrc
rm awscliv2.zip
aws --version
```
`aws-cli/2.15.41 Python/3.11.8 Linux/6.5.0-1018-aws exe/x86_64.ubuntu.22 prompt/off`


# Install virtual env in a project

```bash
pyenv local 3.10.14  # set up the local 
python -m venv .venv
source .venv/bin/activate
pip install --upgrade pip
pip install -r requirements.txt
deactivate
```

to install specific version of a package:
```zsh
pip install example-package==2.0.1
pip install pip-tools
pip freeze > requirements.txt  # better is to use pip-compile
pip-compile requirements.in
pip-compile --output-file dev.txt requirements.in  # if you want different name
```
requirements file uses == for versions

# Encode time zone

```bash
echo "export TZ='PST8PDT'" >> ~/.bashrc
```

## If you must get Conda

Get mamba which is a faster version of `conda` from [here](https://github.com/conda-forge/miniforge#mambaforge):
```zsh
wget https://github.com/conda-forge/miniforge/releases/latest/download/Mambaforge-Linux-x86_64.sh
```
or just get `miniconda`:

```zsh
wget https://repo.anaconda.com/miniconda/Miniconda3-py310_23.3.1-0-Linux-x86_64.sh
```

Install it via:
```bash
source <file you want to run>.sh
```

Update mamba/conda:
```zsh
conda update -n base -c conda-forge conda
```

Init conda:
```zsh
conda init zsh  # Must restart terminal if running this one.
```

### Note for Windows

[Install](https://learn.microsoft.com/en-us/windows/wsl/install) WSL for Windows:
```
wsl --install Ubuntu
```
will need to add both conda and mamba to PATH by editing `~/.bashrc` file and adding these two lines to the end: 
```zsh
source "${CONDA PATH}/etc/profile.d/conda.sh"
source "${CONDA PATH}/etc/profile.d/mamba.sh"
```
Also to set up default terminal in VSCode follow quick instructions [here](https://stuartmccoll.github.io/posts/2021-06-08-set-wsl-as-default-vscode-terminal/).

Windows C hard drive will be by default mounted in `/mnt/c/` folder. 

Use PowerToys/Keyboard Manager to remap the keyboard.

# Create environment

Create environment with `env.yml` or from scratch:
```zsh
conda create -f env.yml -n custom name # notice there is no keyword env
```

```zsh
yes | conda create -n repo_name python=python_ver
conda activate repo_name
deactivate
```

# Environment management

```zsh
conda env list  # list all envs
conda env remove -n ENV_NAME
conda list torch  # lists only package version
```

# Install packages

```zsh
yes | conda install -c conda-forge jupyter_contrib_nbextensions graphviz python-graphviz
```

# Export environment

Alternatively, export environment to a file:
```zsh
conda env export > env.yml
```
or if it is cross-platform export simpler no-build number environment:
```zsh
conda env export --no-builds | grep -v "prefix" > env.yml
```



# Jupyter kernels


Add a kernel:
```zsh
ipython kernel install --name $repo_name --user
```

# Using bash script

- If making shell file make sure it has correct line endings for the system (for example in PyCharm go to File -> File Properties -> Line Separators)

Make sure the file is executable:
```zsh
chmod +x utils/setup_new_project.sh check with ls -l
./utils/setup_new_project.sh <repo_name> <python_version>
# for example ./setup_new_project.sh test_repo 3.10
```

To stop execution after a failure use `|| exit`, for example if the folder doesn't exist following command will stop the script:
```zsh
cd $repo_name || exit
```


# VS Code Extensions

Python, Jupyter, autoDocstring, Code Spell Checker, Dev Containers.